# Algorithmes Génétique et Mémétique
-----------------------------------------------------------

# I. Importations et fonctions communes

## I.1 Importations communes

In [1]:
import job
import ordonnancement
import sommet
import flowshop
import random

In [15]:
N = 70              #Taille de la population
Iteration=15       #Nombre d'itérations maximum pour les algos
iter_max_RL = 50 # Nombre d'itérations maximum pour la recherche locale 

## I.2 Fonctions communes

### I.2.a Initialisation de la population

In [3]:
#Pour générer N ordonnancements
def get_random_population(L, nb_machines) : 
    Pop=[]
    for i in range(N) :
        ordo = ordonnancement.Ordonnancement(nb_machines)
        random.shuffle(L)
        ordo.ordonnancer_liste_job(L)
        Pop+=[ordo]
    return Pop

### I.2.b Score moyen de la population

In [4]:
#Pour calculer la moyenne des solutions
def average_population_grade(Pop):
    somme=0
    for indiv in Pop:
        somme+=indiv.duree
    return somme/len(Pop)

### I.2.c Sélection des parents (méthode de sélection proportionnelle)

In [5]:
def select_parents(population):
    moyenne=average_population_grade(population)
    proba_cumulee=0
    liste_proba_cumulee=[]
    for ordo in population:
        proba_cumulee+=(2-ordo.duree/moyenne)/len(population)*100
        liste_proba_cumulee.append(proba_cumulee)     #Liste des probabilité cumulées de choisir le n'ieme ordonnancement avec une proba de f/f_moyen
    Parents=[]
    for k in range(N):
        proba=random.random()*100
        for k in range(len(population)):       #On parcourt la liste de proba_cumulee jusqu'à ce que la proba aléatoire corresponde à un individu
            if proba<liste_proba_cumulee[k]:
                Parents.append(population[k])
                break
            if proba>=liste_proba_cumulee[len(population)-1]:
                print(proba)
                Parents.append(population[len(population)-1])
                break
    return Parents

### I.2.d Création des couples à partir d'un ensemble de parents

In [6]:
def creation_couples(parents):
    couples=[]
    if len(parents)%2 == 1:
        print("il doit y avoir un nombre pair de parents")
    else:
        for k in range(len(parents)//2):
            couples.append([parents[2*k], parents[2*k+1]])
    return couples

### I.2.e Croisement des deux parents pour donner deux enfants

In [7]:
def croisement(couple):
    pere=couple[0]
    mere=couple[1]
    cassure=len(pere.sequence)//2
    fils=pere.sequence[:cassure]
    fille=mere.sequence[:cassure]
    for k in range(len(pere.sequence)):
        if mere.sequence[k] not in fils:
            fils.append(mere.sequence[k])
        if pere.sequence[k] not in fille:
            fille.append(pere.sequence[k])
    enfant1=ordonnancement.Ordonnancement(pere.nombre_machines)
    enfant1.ordonnancer_liste_job(fils)
    enfant2=ordonnancement.Ordonnancement(pere.nombre_machines)
    enfant2.ordonnancer_liste_job(fille)
    return enfant1, enfant2

### I.2.f Sélection du meilleur individu

In [8]:
def meilleur_individu(population):
    pop_triee= sorted(population, key=lambda ordonnancement: ordonnancement.duree)
    return pop_triee[0]

# II. Algorithme génétique

## II.1 Implémentation

In [9]:
def genetique(L, nb_machine):
    #Creation d'un ensemble d'individu
    population = get_random_population(L,nb_machine)                        #population est une liste d'ordonnancements aléatoires
    average_grade = average_population_grade(population)
    print('Starting grade: ', average_grade)
    i=0
    while i < Iteration :
        #Affichage de la moyenne
        average_grade = average_population_grade(population)
        print(i, 'Current grade: ', average_grade)
        
        #Choix de N parents / Un individu peut être parent plusieurs fois
        Parents=select_parents(population)
        
        #Creation de N/2 couples de parents
        Couples=creation_couples(Parents)
    
        #Croisement
        nouvelle_generation=[]
        for couple in Couples:
            enfant1, enfant2=croisement(couple)
            nouvelle_generation.append(enfant1)
            nouvelle_generation.append(enfant2)
        
        population = nouvelle_generation
        i+=1
    return meilleur_individu(population)

## II.2 Tests

In [24]:
print("JEU 1 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('data/jeu1.txt')
print('Meilleur individu :',genetique(prob.liste_jobs, prob.nombre_machines).duree)

JEU 1 :
Starting grade:  58.08
0 Current grade:  58.08
1 Current grade:  58.25
2 Current grade:  58.41
3 Current grade:  58.08
4 Current grade:  58.15
5 Current grade:  57.7
6 Current grade:  57.39
7 Current grade:  57.58
8 Current grade:  57.27
9 Current grade:  58.05
Meilleur individu : 54


In [25]:
print("JEU 2 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('data/jeu2.txt')
print('Meilleur individu :',genetique(prob.liste_jobs, prob.nombre_machines).duree)

JEU 2 :
Starting grade:  794.17
0 Current grade:  794.17
1 Current grade:  799.06
2 Current grade:  800.1
3 Current grade:  800.85
4 Current grade:  797.56
5 Current grade:  798.21
6 Current grade:  796.74
7 Current grade:  799.54
8 Current grade:  796.16
9 Current grade:  794.84
Meilleur individu : 730


In [26]:
print("Tai01 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('data/jeu_de_données_1/tai01.txt')
print('Meilleur individu :',genetique(prob.liste_jobs, prob.nombre_machines).duree)


Tai01 :
Starting grade:  1514.33
0 Current grade:  1514.33
1 Current grade:  1507.21
2 Current grade:  1506.8
3 Current grade:  1501.28
4 Current grade:  1498.83
5 Current grade:  1507.0
6 Current grade:  1489.38
7 Current grade:  1492.48
8 Current grade:  1484.68
9 Current grade:  1489.64
Meilleur individu : 1324


# III. Algorithme mémétique

## III.1 Fonctions supplémentaires

### III.1.a Recherche locale en échangeant deux jobs aléatoirement

In [10]:
def RL(individu):
    i=0
    while i<iter_max_RL:
        sequence=individu.sequence
        x1=random.randint(0,len(sequence)-1)
        x2=random.randint(0,len(sequence)-1)
        while x2==1:
            x2=random.randint(0,len(sequence)-1)
        y1=sequence[x1]
        y2=sequence[x2]
        sequence[x1]=y2
        sequence[x2]=y1
        new=ordonnancement.Ordonnancement(individu.nombre_machines)
        new.ordonnancer_liste_job(sequence)
        if new.duree<individu.duree:
            individu=new
        i+=1
    return individu

### III.1.c Remplacement

In [11]:
def remplacement(population, new_generation):
    pop=population+new_generation
    pop_triee= sorted(pop, key=lambda ordonnancement: ordonnancement.duree)
    return pop_triee[:len(population)]

## III.2 Implémentation

In [12]:
import time

def memetique(L, nb_machine):
    # Time
    start_time = time.time()    
    #Creation d'un ensemble d'individu
    population = get_random_population(L,nb_machine)                        #population est une liste d'ordonnancements aléatoires
    average_grade = average_population_grade(population)
    print('Starting grade: ', average_grade)
    print('-------------------------------')
    i=0
    while i < Iteration :
        #Affichage de la moyenne
        average_grade = average_population_grade(population)
        print(i, 'Current grade: ', average_grade)
        
        #Choix de N parents / Un individu peut être parent plusieurs fois
        Parents=select_parents(population)
        
        #Creation de N/2 couples de parents
        Couples=creation_couples(Parents)
    
        #Croisement
        nouvelle_generation=[]
        for couple in Couples:
            enfant1, enfant2=croisement(couple)
            enfant1=RL(enfant1)
            enfant2=RL(enfant2)
            nouvelle_generation.append(enfant1)
            nouvelle_generation.append(enfant2)
        
        population = remplacement(population,nouvelle_generation)
        i+=1
    elapsed_time = time.time() - start_time
    print('----------- TIME : {}s ----------- '.format(round(elapsed_time,3)))
    return meilleur_individu(population)

## III.3 Tests

In [13]:
print("JEU 1 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('data/jeu1.txt')
print('Meilleur individu :',memetique(prob.liste_jobs, prob.nombre_machines).duree)

JEU 1 :
Starting grade:  58.32
-------------------------------
0 Current grade:  58.32
1 Current grade:  54.0
2 Current grade:  54.0
3 Current grade:  54.0
4 Current grade:  54.0
5 Current grade:  54.0
6 Current grade:  54.0
7 Current grade:  54.0
8 Current grade:  54.0
9 Current grade:  54.0
10 Current grade:  54.0
11 Current grade:  54.0
12 Current grade:  54.0
13 Current grade:  54.0
14 Current grade:  54.0
----------- TIME : 4.136s ----------- 
Meilleur individu : 54


In [44]:
print("JEU 2 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('data/jeu2.txt')
print('Meilleur individu :',memetique(prob.liste_jobs, prob.nombre_machines).duree)

JEU 2 :
Starting grade:  797.11
-------------------------------
0 Current grade:  797.11
1 Current grade:  725.57
2 Current grade:  717.87
3 Current grade:  714.94
4 Current grade:  713.11
5 Current grade:  711.98
6 Current grade:  710.95
7 Current grade:  710.24
8 Current grade:  709.87
9 Current grade:  709.27
----------- TIME : 2.954s ----------- 
Meilleur individu : 704


In [14]:
print("tai11 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('/Users/corentin/Desktop/Cours/2A/programmation_dynamique/projet/data/jeu_de_données_1/tai11.txt')
L = prob.liste_jobs
m = prob.nombre_machines


start_time = time.time()
result=memetique(prob.liste_jobs, prob.nombre_machines)
elapsed_time = time.time() - start_time
print('----- RESULTAT-----')
print('Temps de résolution : {}s'.format(round(elapsed_time,3)))
print("durée :", result.duree)
print([ job.numero for job in result.sequence])

tai11 :
Starting grade:  2032.21
-------------------------------
0 Current grade:  2032.21
1 Current grade:  1863.65
2 Current grade:  1839.78
3 Current grade:  1827.01
4 Current grade:  1821.8
5 Current grade:  1812.73
6 Current grade:  1807.89
7 Current grade:  1805.04
8 Current grade:  1803.44
9 Current grade:  1798.89
----------- TIME : 22.102s ----------- 
----- RESULTAT-----
Temps de résolution : 22.103s
durée : 1717
[11, 7, 5, 6, 19, 15, 10, 8, 16, 14, 12, 2, 4, 0, 9, 1, 3, 17, 13, 18]


In [18]:
print("tai51 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('/Users/corentin/Desktop/Cours/2A/programmation_dynamique/projet/data/jeu_de_données_1/tai51.txt')
L = prob.liste_jobs
m = prob.nombre_machines
# 3874

start_time = time.time()
result=memetique(prob.liste_jobs, prob.nombre_machines)
elapsed_time = time.time() - start_time
print('----- RESULTAT-----')
print('Temps de résolution : {}s'.format(round(elapsed_time,3)))
print("durée :", result.duree)
print([ job.numero for job in result.sequence])

tai51 :
Starting grade:  4863.15
-------------------------------
0 Current grade:  4863.15
1 Current grade:  4640.45
2 Current grade:  4603.25
3 Current grade:  4585.51
4 Current grade:  4570.97
5 Current grade:  4563.51
6 Current grade:  4558.84
7 Current grade:  4551.07
8 Current grade:  4547.4
9 Current grade:  4544.86
10 Current grade:  4541.01
11 Current grade:  4538.39
12 Current grade:  4536.91
13 Current grade:  4534.77
14 Current grade:  4533.36
----------- TIME : 152.433s ----------- 
----- RESULTAT-----
Temps de résolution : 152.433s
durée : 4411
[24, 42, 35, 32, 20, 6, 1, 10, 5, 45, 47, 43, 16, 19, 22, 44, 41, 14, 15, 0, 48, 13, 27, 40, 38, 36, 30, 28, 17, 34, 23, 33, 31, 4, 3, 39, 18, 26, 2, 25, 46, 29, 21, 7, 11, 9, 49, 8, 12, 37]


In [16]:
print("tai52 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('/Users/corentin/Desktop/Cours/2A/programmation_dynamique/projet/data/jeu_de_données_2/tai52.txt')
L = prob.liste_jobs
m = prob.nombre_machines
# 3874
# Doucement mais surement ! 

start_time = time.time()
result=memetique(prob.liste_jobs, prob.nombre_machines)
elapsed_time = time.time() - start_time
print('----- RESULTAT-----')
print('Temps de résolution : {}s'.format(round(elapsed_time,3)))
print("durée :", result.duree)
print([ job.numero for job in result.sequence])

tai52 :
Starting grade:  4709.742857142857
-------------------------------
0 Current grade:  4709.742857142857
1 Current grade:  4528.585714285714
2 Current grade:  4476.028571428571
3 Current grade:  4456.828571428571
4 Current grade:  4443.4
5 Current grade:  4432.2
6 Current grade:  4420.4857142857145
7 Current grade:  4412.2
8 Current grade:  4405.414285714286
9 Current grade:  4398.9857142857145
10 Current grade:  4394.5
11 Current grade:  4387.9857142857145
12 Current grade:  4384.471428571429
13 Current grade:  4382.928571428572
14 Current grade:  4375.957142857143
----------- TIME : 46.421s ----------- 
----- RESULTAT-----
Temps de résolution : 46.421s
durée : 4265
[36, 37, 5, 23, 44, 16, 43, 30, 28, 22, 35, 6, 49, 0, 33, 24, 21, 2, 40, 12, 34, 41, 46, 38, 39, 7, 45, 29, 17, 1, 42, 10, 27, 20, 8, 32, 26, 31, 3, 9, 11, 13, 25, 14, 19, 4, 48, 15, 18, 47]
